In [43]:
import os
import reservoirpy as rpy
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.metrics import r2_score
from reservoirpy.nodes import Reservoir, Ridge
import logging
import gc
import pickle

# ログ設定
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# グローバル設定
rpy.verbosity(0)  # Verboseモードを無効
rpy.set_seed(42)  # 再現性のためのシード設定

In [44]:
# パラメータ設定
n_timesteps = 5000
units_list = 100 * 2 ** np.arange(0, 6)  # 100, 200, 400, ..., 3200
spectral_radii = np.linspace(0.1, 1.5, 10)  # スペクトル半径の範囲
max_delay = 100  # 遅延数
batch_size = 20  # バッチサイズを最適化

# 結果保存先のディレクトリ
output_dir = "jupyter/src/memory_capacity/results"
os.makedirs(output_dir, exist_ok=True)  # ディレクトリがなければ作成

In [45]:
# データ生成
np.random.seed(42)
input_signal = np.random.randn(n_timesteps).reshape(-1, 1)
input_signal = (input_signal - np.mean(input_signal)) / np.std(input_signal)
X = np.array([input_signal[:-d] for d in range(1, max_delay + 1)], dtype=object)
Y = np.array([input_signal[d:] for d in range(1, max_delay + 1)], dtype=object)

In [46]:
# ESN（Echo State Network）を作成
def create_standard_esn(sr, units):
    logging.info(f"Creating ESN with spectral radius={sr} and units={units}")
    reservoir = Reservoir(units=units, input_scaling=1, lr=1, sr=sr)
    readout = Ridge(output_dim=1, ridge=1e-5)
    return reservoir >> readout

In [47]:
# R^2スコアを計算
def compute_r2(spectral_radius, units, input_signal, output_signal):
    esn = create_standard_esn(spectral_radius, units)
    esn.fit(input_signal, output_signal)
    pred = esn.run(input_signal)
    r2 = r2_score(output_signal, pred)
    del esn, pred  # 不要な変数を削除してメモリ解放
    gc.collect()   # ガベージコレクションを実行
    return max(r2, 0)  # R^2スコアが負の場合は0にする

# メモリ容量を計算
def compute_memory_capacity(spectral_radius, units, X, Y):
    logging.info(f"Calculating memory capacity for spectral radius={spectral_radius} and units={units}")
    
    r2_scores = []
    for start in range(0, max_delay, batch_size):
        end = min(start + batch_size, max_delay)
        r2_batch = Parallel(n_jobs=8, backend="loky")(delayed(compute_r2)(
            spectral_radius, units, X[d], Y[d]) for d in range(start, end))
        r2_scores.extend(r2_batch)
        # バッチ毎にメモリ解放
        gc.collect()
    
    memory_capacity = np.sum(r2_scores)
    
    # 結果をDataFrameにまとめる
    results_df = pd.DataFrame({
        "Spectral Radius": [spectral_radius],
        "Units": [units],
        "Memory Capacity": [memory_capacity],
        "R2 Scores": [r2_scores]
    })


    # CSVファイルに結果を保存
    save_path = os.path.join(output_dir, "units_MC.csv")
    header = not os.path.exists(save_path)  # ファイルが存在しない場合のみヘッダーを追加
    results_df.to_csv(save_path, mode="a", header=header, index=False)
    
    # Pickleでも結果を保存
    pickle_save_path = os.path.join(output_dir, f"memory_capacity_sr_{spectral_radius}_units_{units}.pkl")
    with open(pickle_save_path, "wb") as f:
        pickle.dump(results_df, f)
    
    logging.info(f"Memory capacity for spectral radius={spectral_radius}, units={units}: {memory_capacity}")
    return memory_capacity

# スペクトル半径ごとにメモリ容量を計算
def compute_memory_capacity_for_spectral_radius(spectral_radius, units):
    return compute_memory_capacity(spectral_radius, units, X, Y)

In [ ]:
# 実行例
test_result = compute_memory_capacity_for_spectral_radius(spectral_radius=0.5, units=100)
print(f"Test memory capacity result: {test_result}")

2024-11-05 17:11:47,783 - INFO - Calculating memory capacity for spectral radius=0.5 and units=100
